# 自動填入google表單

## 方法1. 用selenium來填入選項與click button

In [14]:
import re
import time
import random
import numpy as np
import requests as rq
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType

### 用selenium開啟網頁

In [15]:
spyderUrl = "https://docs.google.com/forms/d/e/********************/viewform"
# 偽裝用
# headers = {
#     "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
#     "Accept-Encoding": "gzip, deflate, br", 
#     "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7", 
#     "Host": "docs.google.com",  #目標網站 
#     "Sec-Fetch-Dest": "document", 
#     "Sec-Fetch-Mode": "navigate", 
#     "Sec-Fetch-Site": "none", 
#     "Upgrade-Insecure-Requests": "1", 
#     "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_4_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.86 Safari/537.36", #使用者代理
#     "Referer": "https://www.google.com/"
# }

options = Options()
options.add_argument("--disable-notifications")

# 若遇到錯誤 selenium.common.exceptions.WebDriverException: Message: unknown error: cannot find Chrome binary
# 要設定binary位置
options.binary_location = "/Users/yihsiu/Desktop/Google Chrome.app/Contents/MacOS/Google Chrome"

chrome = webdriver.Chrome('./chromedriver', options=options)
chrome.get(spyderUrl)

### 用selenium填入項目

In [11]:
element = chrome.find_elements_by_css_selector('input.quantumWizTextinputPaperinputInput.exportInput')
print(element[0])
element[0].send_keys("測試填入姓名")
element[1].send_keys("填入電話")

# 選項有三個 看要按哪個0~2
radioButton = chrome.find_elements_by_css_selector('label.docssharedWizToggleLabeledContainer.freebirdFormviewerComponentsQuestionRadioRadio')
radioButton[1].click()

<selenium.webdriver.remote.webelement.WebElement (session="943e4f66c1d778f3a0b4fdbfbf6052be", element="025a7f90-7721-471f-8e0e-1c694dd45dd0")>


In [12]:
# click送出button
sendButton = chrome.find_elements_by_css_selector("span.appsMaterialWizButtonPaperbuttonContent.exportButtonContent")
sendButton[0].click()

## 方法2. 利用beautifulSoup + selenium 爬蟲爬出entry.id 然後根據post要的項目填入後直接送出

In [16]:
html = chrome.page_source
soup = BeautifulSoup(html, 'html.parser')
getInput = soup.find_all("input",type="hidden")
print(getInput)
# 取出 name值
finalName = []
for name in getInput:
    getName = name.attrs["name"]
    if getName[0:5] == "entry":
        finalName.append(getName.replace("_sentinel",""))
        
print("取出的值：",finalName)

[<input name="entry.59437064" type="hidden" value=""/>, <input name="entry.2027355597" type="hidden" value=""/>, <input jsname="DTMEae" name="entry.716179405_sentinel" type="hidden"/>, <input name="fvv" type="hidden" value="1"/>, <input name="draftResponse" type="hidden" value='[null,null,"-6274069285456166073"]
'/>, <input name="pageHistory" type="hidden" value="0"/>, <input name="fbzx" type="hidden" value="-6274069285456166073"/>]
取出的值： ['entry.59437064', 'entry.2027355597', 'entry.716179405']


In [17]:
# 發送出去的值
inputData = {
#     "entry.59437064": "bb",
#     "entry.2027355597": "me",
#     "entry.716179405": "選項 1",
#     "entry.716179405_sentinel":"選項 1",
    "fvv": "0",
    "draftResponse": "[]",
    "pageHistory": "0",
    "fbzx": "915303027356312637"
}
print(inputData)

# 根據取出的值放入要發送的物件中
inputData[finalName[0]] = "姓名叫啥"
inputData[finalName[1]] = "電話000000"
inputData[finalName[2]] = "選項 2"

print(inputData)
# url = 'https://docs.google.com/forms/d/e/********************/formResponse'
url = "https://docs.google.com/forms/d/e/********************/formResponse"

try:
    res = rq.post(url, data=inputData)
    res.raise_for_status()
    if res.status_code == 200 :
        print('填成功了 ' )
except requsets.HTTPError:
    print('HTTP Error!')
    
    num -= 1

{'fvv': '0', 'draftResponse': '[]', 'pageHistory': '0', 'fbzx': '915303027356312637'}
{'fvv': '0', 'draftResponse': '[]', 'pageHistory': '0', 'fbzx': '915303027356312637', 'entry.59437064': '姓名叫啥', 'entry.2027355597': '電話000000', 'entry.716179405': '選項 2'}
填成功了 
